# Another 1D GP Example

We generate realizations of a simple one dimensional Gaussian Process with zero mean and Matern covariance. Then we use the LaplaceInterpolation procedure to produce the interpolation.

In [ ]:
using GaussianProcesses, LaplaceInterpolation, Random, LinearAlgebra, Plots
Random.seed!(20140430);

In [ ]:
function get_gp(ν, ll, lσ)
    kern = Matern(ν, ll, lσ)                   
    # We will use no observation noise here for the purposes of generating data
    return GP(x, y, MeanZero(), kern)
end

function gprealiz(x, y, ν, ll, lσ, nrealiz = 1)
    kern = Matern(ν, ll, lσ)                   
    # We will use no observation noise here for the purposes of generating data
    gp = GP(x, y, MeanZero(), kern)
    # Generate realizations of the data. It is given as μ + σLV, where L is the Cholesky factor of K
    # and V is drawn from an iid N(0,1) distribution
    μ, Σ = predict_f(gp, collect(1.0:m), full_cov = true)
    L = cholesky(Σ)
    realiz = zeros(m, nrealiz)
    for i in 1:nrealiz
        realiz[:, i] = μ + L.L * randn(m)
    end
    return realiz
end

In [ ]:
# Training data
n = 10
m = 2 * n
v = Float64.(randperm(m))
x = sort(v[1:n])
y = sin.(2π * x * 0.1) + 0.05*randn(n)

# The Matern parameters are set on a log scale
ν = 1.5
ll = 0.5
lσ = 0.0 #0.05
nrealiz = 100

realiz = gprealiz(x, y, ν, ll, lσ, nrealiz)

In [ ]:
gp = get_gp(ν, ll, lσ)
plot(gp; xlabel="x", ylabel="y", title="Gaussian process", legend=true, label = ["μ" "y"],
        xlim = [0,21]) 
plot!(realiz, c = RGBA(0,0,0,0.1), label = "")

In [ ]:
h = k = 1.0
discard = Int64.(v[(n+1):m])

y_lap = matern_1d_grid(y, discard, 1, 0.0, h)
d = 1
m = 2 # nu + d/2
# m = dimension/2+nu
# epsilon = sqrt(2*nu)/length_scale
epsilon = sqrt(3)/exp(ll)

y_mat = matern_1d_grid(y, discard, m, epsilon, h, k)